# ShowCase Exapmles
this is a file that contains codes that were removed but are importat, since they were used to test some theories

In [33]:
from utils.utils import *
from utils.ee_utils import *
from utils.utils import TextColors as tc
import ee
import geemap
from rois import ROIS
rois = ROIS()

In [2]:
ee.Initialize()
Map = geemap.Map()

## High cloud cover results in false snow_cover result in sentinel 2 images

For exapmle in this case, we are testing an image in summer in paris, but as you can see we are seeing a high snow/ice coverege and that is because there is a high snow covege.

In [36]:
roi = rois.bercy

s2_test_col = ee.ImageCollection('COPERNICUS/S2_SR') \
                    .filterDate('2021-05-17', '2021-05-19') \
                    .filterBounds(roi)
                    
sen2_print(s2_test_col)
s2_test_img = s2_test_col.first()

scl = s2_test_img.select('SCL')
mask = scl.eq(11)

Map = geemap.Map()
Map.addLayer(s2_test_img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 Image', True)
Map.addLayer(mask.updateMask(mask), {'palette': 'blue'}, 'Water Mask')
Map.centerObject(roi, zoom=10)
Map.addLayerControl()
Map


system:time_start---------> ['2021-05-18']
roi_cloud_cover-----------> []
CLOUDY_PIXEL_PERCENTAGE---> [70.48862]
CLOUD_SHADOW_PERCENTAGE---> [0.703129]
VEGETATION_PERCENTAGE-----> [12.594715]
NOT_VEGETATED_PERCENTAGE--> [1.311284]
CLOUD_COVERAGE_ASSESSMENT-> [70.48862]
GENERATION_TIME-----------> [1621347252000]
SENSING_ORBIT_NUMBER------> [94]
NODATA_PIXEL_PERCENTAGE---> [26.389542]
DATATAKE_TYPE-------------> ['INS-NOBS']
SENSING_ORBIT_NUMBER------> [94]
SNOW_ICE_PERCENTAGE-------> [9.749825]
THIN_CIRRUS_PERCENTAGE----> [2.360785]
WATER_PERCENTAGE----------> [0.127259]
system:band_names---------> [['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']]


Map(center=[48.843641547684896, 2.380256652833762], controls=(WidgetControl(options=['position', 'transparent_…

## Sentinel 1A and 1B, Tandem Orbit 
sentinel 1a and 1b orbits result in a pretty simmilar, becuse of the short Spatial BaseLine.

- Although I still don't know, how can you determine whether two, realtive orbit numbers are tandem or not. -> <span style="color: #FF1111;">More Research Needed</span>

### Hmmm Interesting...

Well, Look ate the dates `AB BA AB BA ...` and each with `1 day` diffrence, and more interesting, 

In [80]:
def ee_property_printer(s1_collection, propertie_name_list = ['system:time_start','orbitProperties_pass',
                                                     'resolution','resolution_meters','platform_number',
                                                     'productType','orbitNumber_start','orbitNumber_stop',
                                                     'transmitterReceiverPolarisation','system:band_names','instrumentMode',
                                                     'relativeOrbitNumber_stop','relativeOrbitNumber_start','cycleNumber'],first_is_t_in_millis= True):
    """
    A function that prints the properties EarthEngine Image.

    Parameters:
    -----------
    s1_collection : ee.ImageCollection
        The Sentinel-1 image collection whose properties are to be printed.
    propertie_name_list : list of str, optional (default=['system:time_start', 'orbitProperties_pass', 'resolution', 'resolution_meters', 'platform_number', 'productType', 'orbitNumber_start', 'orbitNumber_stop', 'transmitterReceiverPolarisation', 'system:band_names', 'instrumentMode', 'relativeOrbitNumber_stop', 'relativeOrbitNumber_start', 'cycleNumber'])
        A list of property names to be printed for each image in the collection.
    first_is_t_in_millis : bool, optional (default=True)
        A flag indicating whether the first property in `property_name_list` represents a timestamp in milliseconds.
        If True, the timestamp will be converted to a human-readable date string before being printed.

    Returns:
    --------
    None

    Example:
    --------
    >>> collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2019-01-01', '2019-01-31').filterBounds(geometry)
    >>> sen1_print(collection, propertie_name_list=['system:time_start', 'relativeOrbitNumber_start', 'transmitterReceiverPolarisation'], first_is_t_in_millis=True)
    system:time_start -> [datetime.datetime(2019, 1, 1, 5, 42, 36, 196000), datetime.datetime(2019, 1, 1, 5, 42, 51, 427000), ...]
    relativeOrbitNumber_start -> [17, 35, ...]
    transmitterReceiverPolarisation -> ['VV', 'VV', ...]
    """

    if first_is_t_in_millis:
            agg_list = [s1_collection.aggregate_array(propertie_name_list[0]).map(millis_to_date_string)]
    else:
        agg_list = [s1_collection.aggregate_array(propertie_name_list[0])]
        
    for propertie_name in propertie_name_list[1:]:
        agg_list.append(s1_collection.aggregate_array(propertie_name))
        
    ee_list =ee.List(agg_list)
    

    properties_list = ee_list.getInfo()
    
    df = pd.DataFrame(properties_list)
    df.insert(0, 'Property', propertie_name_list)
    styled_df = df.style.set_properties(**{'text-align': 'left'})
    styled_df = styled_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    return styled_df

In [81]:
# Overwrite the default print function to for a consicise print
sen1_print = lambda s2_collection: ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                                    'relativeOrbitNumber_start', 'orbitProperties_pass',
                                                                                                    'cycleNumber'], first_is_t_in_millis=True)

In [51]:
roi = rois.paris
s2_col,s1_col = s1s2(roi,date = ('2021-07-15', '2021-08-15'),priority_path = 'DESCENDING',max_cloud = 100,max_snow = 100,month_span = 1,retry_days=0)

◍◍Finding S2
◍Single scene coverege was fount!
◍◍Finding S1
mean date:  2021-07-21
final date range:  2021-06-21 2021-08-21
Snowy days       :  []
Snowy days Buffed:  []
◍checking for DESCENDING single scene
Mosiac Covers 93.96914404374861 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
collection size before removing snowy dates:  20
collection size after removing snowy dates:  20
◍Collection Found!


In [82]:
sen1_print(s1_col.sort('system:time_start'))

,Property,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,system:time_start,2021-06-25,2021-06-26,2021-07-01,2021-07-02,2021-07-07,2021-07-08,2021-07-13,2021-07-14,2021-07-19,2021-07-20,2021-07-25,2021-07-26,2021-07-31,2021-08-01,2021-08-06,2021-08-07,2021-08-12,2021-08-13,2021-08-18,2021-08-19
1,platform_number,B,A,A,B,B,A,A,B,B,A,A,B,B,A,A,B,B,A,A,B
2,relativeOrbitNumber_start,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,234,235,164,165,235,236,165,166,236,237,166,167,237,238,167,168,238,239,168


In [79]:
sen1_print(s1_col.filter(ee.Filter.eq('platform_number', 'A')).sort('system:time_start'))

TypeError: set_table_styles() got an unexpected keyword argument 'text-align'

In [72]:
sen1_print(s1_col.filter(ee.Filter.eq('platform_number', 'B')).sort('system:time_start'))

,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-25,2021-07-02,2021-07-07,2021-07-14,2021-07-19,2021-07-26,2021-07-31,2021-08-07,2021-08-12,2021-08-19
1,platform_number,B,B,B,B,B,B,B,B,B,B
2,relativeOrbitNumber_start,8,110,8,110,8,110,8,110,8,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,164,165,165,166,166,167,167,168,168


In [73]:
print(tc.BOLD,'Relative orbit number 8: ',tc.ENDC)
sen1_print(s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(8))).sort('system:time_start'))
print(tc.BOLD,'Relative orbit number 110: ',tc.ENDC)
sen1_print(s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(110))).sort('system:time_start'))

 Relative orbit number 8:  
 Relative orbit number 110:  


,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-26,2021-07-02,2021-07-08,2021-07-14,2021-07-20,2021-07-26,2021-08-01,2021-08-07,2021-08-13,2021-08-19
1,platform_number,A,B,A,B,A,B,A,B,A,B
2,relativeOrbitNumber_start,110,110,110,110,110,110,110,110,110,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,234,164,235,165,236,166,237,167,238,168


In [19]:
vis_params_s1_bright = {
    'min': -10,
    'max': 10,
    'bands': ['VV_db']
}

In [20]:
s1_col = s1_col.map(toLinear)

In [21]:
s1a = toDb(s1_col.filter(ee.Filter.eq('platform_number', 'A')).mean().clip(roi))
s1b = toDb(s1_col.filter(ee.Filter.eq('platform_number', 'B')).mean().clip(roi))
left_layer = geemap.ee_tile_layer(s1a, vis_params_s1_bright, 'S1a')
right_layer = geemap.ee_tile_layer(s1b, vis_params_s1_bright, 'S1b')

In [22]:
s1a.propertyNames().getInfo()

['system:footprint', 'system:bands', 'system:band_names']

In [9]:
Map = geemap.Map(zoom=12)
Map.centerObject(roi)
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…